In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
import xgboost
import lightgbm as lgb

from sklearn import linear_model
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.metrics import explained_variance_score, mean_absolute_error
import matplotlib.pyplot as plt

%matplotlib inline

# load data

In [2]:
df_properties_2016 = pd.read_csv('../data/properties_2016.csv')

/Users/dai_li/Workspace/VirtualEnv/competition/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
df_transactions_2016 = pd.read_csv('../data/train_2016_v2.csv', parse_dates=['transactiondate'])

In [8]:
df_properties = df_properties_2016
df_transactions = df_transactions_2016

In [156]:
df = pd.merge(left=df_properties, right=df_transactions, on='parcelid', how= 'left')

# Pre-process data

## define fields

In [ ]:
# lets make this assumption -- all the features are generated per monthly basis
# lets assume we can use future information - use the log error to generate some features

## categorical features

In [157]:
# add censustract
t = pd.DataFrame(df['rawcensustractandblock'].astype(str).str.split('.',1).tolist(),columns = ['censustrack','censusblock'])
df['censustrack'] = t['censustrack']

t = pd.DataFrame(df_properties['rawcensustractandblock'].astype(str).str.split('.',1).tolist(),columns = ['censustrack','censusblock'])
df_properties['censustrack'] = t['censustrack']


In [158]:
###################################
####### DIMENSION: SPACE ##########
###################################

parcel_location_variables = ['regionidneighborhood','regionidzip', 'regionidcity', 'censustrack','rawcensustractandblock']

# iterate through all above regions
for region in parcel_location_variables:
    
    #### COUNT OF PROPERTIES ####
    # number of properties in the zipcode
    df['f_num_n_prop_in_'+region] = df[region].map(df_properties[region].value_counts().to_dict())
    
    ##### HOW NEW IS THIS BUILDING COMPARING TO OTHER BUILDINGS #####
    df['f_cat_median_year_in_'+region] = df[region].map(df_properties.groupby(region)['yearbuilt'].aggregate('median').to_dict())
    df['f_num_how_new_in_'+region] = df['yearbuilt'] - df['f_cat_median_year_in_'+region]

    # Neighborhood latitude and longitude
    df['f_num_median_lat_in_'+region] = df[region].map(df_properties.groupby(region)['latitude'].aggregate('median').to_dict())
    df['f_num_median_lon_in_'+region] = df[region].map(df_properties.groupby(region)['longitude'].aggregate('median').to_dict())

    #### TRANSACTION RELATED ####
    # how many transaction made in this region per observed properties
    # this tells us how active a region is
    df['f_num_pct_trans_in_'+region] = df.groupby(region)['logerror'].aggregate('count') * 1.0 / df['f_num_n_prop_in_'+region] 

    #### LOG ERROR RELATED ####
    df['f_num_error_std_in_'+region] = df[region].map(df[df.transactiondate < '2016-10-01'].groupby(region)['logerror'].aggregate("std").to_dict())
    df['f_num_error_mean_in_'+region] = df[region].map(df[df.transactiondate < '2016-10-01'].groupby(region)['logerror'].aggregate("mean").to_dict())
    
    

In [144]:
###################################
####### DIMENSION: TIME ###########
###################################

time_windows = [30,60,90,180]

for time_window in time_windows:
    #### TRANSACTION RELATED FEATURES #####
    # I assume in the actual model supply/demand should also be used
    # now I will try to use historical transaction data to mimic demand
    # what is this value comparing to past few months?
    
    #### LOG ERROR RELATED FEATURES ####
    # what is the corresponding logerror rate?
    
    #### LOGERROR RATE FOR CATEGORICAL VARIABLES #####
    # assumption is that model error should persist for same type of properties # 
    # now this is tricky - logerror is not taken into consideration in the actual model
    # so this may end up being useless
    # but probably worth confirming. we will see
    const_numerator = 5
    const_denominator = 5

    #### LOGERROR RATE FOR CONTINUOUS VARIABLE BINS #####
    # assump is that model error should persist for the same type of properties # 
    # same to above #
    
    # ***** generate absolute and relative error for each month **** ### 
    
    
    ### generate seasonality features ####
    # day of week
    # week of month
    # quarter of year (only for 2017)


In [163]:
df.groupby('transactiondate')['transactiondate'].aggregate('count').rolling(window = 60,min_periods=0).mean().plot()

KeyboardInterrupt: 

In [ ]:
#### LOG ERROR RELATED TERMS ####


In [ ]:
###################################
#######  ###########
###################################
  

In [161]:
df.transactiondate.head()

0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: transactiondate, dtype: datetime64[ns]

# pickle df

In [160]:
df.to_csv('../tmp/processed_df_2016.csv', index=False, header=True)

In [159]:
pickle.dump(df, open('../tmp/processed_df_2016.pkl', 'w'))

# Test loading and scoring in model